In [ ]:
import os
import shutil
import warnings

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from tqdm import tqdm

import open3d as o3d
from plyfile import PlyData

# Import point cloud transformations and utilities from openpoints
from openpoints.transforms import (
    Compose, PointCloudXYZAlign, RandomHorizontalFlip, PointCloudScaling,
    PointCloudTranslation, PointCloudJitter, PointCloudCenterAndNormalize, PointCloudToTensor
)
from openpoints.utils import ConfusionMatrix

# Import the Lion optimizer
from lion_pytorch import Lion  

# Ignore warnings for clean output
warnings.filterwarnings("ignore")

# Define constants for segmentation
NUM_CLASSES = 41
SMOOTH_FACTOR = 0.1

# Define class labels
CLASS_LABELS = (
    "wall", "floor", "cabinet", "bed", "chair", "sofa", "table", "door", "window", "bookshelf",
    "picture", "counter", "desk", "curtain", "refrigerator", "shower curtain", "toilet", "sink",
    "bathtub", "otherfurniture",
)

# Define the color map for scanNet
SCANNET_COLOR_MAP = {
    0: (0.0, 0.0, 0.0),
    1: (174.0, 199.0, 232.0),
    2: (152.0, 223.0, 138.0),
    3: (31.0, 119.0, 180.0),
    4: (255.0, 187.0, 120.0),
    5: (188.0, 189.0, 34.0),
    6: (140.0, 86.0, 75.0),
    7: (255.0, 152.0, 150.0),
    8: (214.0, 39.0, 40.0),
    9: (197.0, 176.0, 213.0),
    10: (148.0, 103.0, 189.0),
    11: (196.0, 156.0, 148.0),
    12: (23.0, 190.0, 207.0),
    14: (247.0, 182.0, 210.0),
    15: (66.0, 188.0, 102.0),
    16: (219.0, 219.0, 141.0),
    17: (140.0, 57.0, 197.0),
    18: (202.0, 185.0, 52.0),
    19: (51.0, 176.0, 203.0),
    20: (200.0, 54.0, 131.0),
    21: (92.0, 193.0, 61.0),
    22: (78.0, 71.0, 183.0),
    23: (172.0, 114.0, 82.0),
    24: (255.0, 127.0, 14.0),
    25: (91.0, 163.0, 138.0),
    26: (153.0, 98.0, 156.0),
    27: (140.0, 153.0, 101.0),
    28: (158.0, 218.0, 229.0),
    29: (100.0, 125.0, 154.0),
    30: (178.0, 127.0, 135.0),
    32: (146.0, 111.0, 194.0),
    33: (44.0, 160.0, 44.0),
    34: (112.0, 128.0, 144.0),
    35: (96.0, 207.0, 209.0),
    36: (227.0, 119.0, 194.0),
    37: (213.0, 92.0, 176.0),
    38: (94.0, 106.0, 211.0),
    39: (82.0, 84.0, 163.0),
    40: (100.0, 85.0, 144.0)
}

# Define the ScanNet dataset with point resizing and optional normalization
class ScannetDataset(Dataset):
    def __init__(self, root='/kaggle/input/scannet/scans', is_train=True, transform=None, target_size=10000):

        self.root = root
        self.files = os.listdir(root)
        length = len(self.files)
        split_idx = int(0.9 * length)
        self.files = self.files[:split_idx] if is_train else self.files[split_idx:]
        self.transform = transform
        self.target_size = target_size

    def __len__(self):
        return len(self.files)

    def resize_point_cloud(self, points, labels):
  
        num_points = points.shape[0]
        # Select one point per unique label
        unique_labels = np.unique(labels)
        selected_indices = [np.random.choice(np.where(labels == label)[0]) for label in unique_labels]
        selected_indices = np.array(selected_indices)
        
        if num_points > self.target_size:
            # Fill remaining points with random indices (avoiding duplicates)
            remaining_indices = np.setdiff1d(np.arange(num_points), selected_indices)
            random_indices = np.random.choice(
                remaining_indices, self.target_size - len(selected_indices), replace=False
            )
            all_indices = np.concatenate([selected_indices, random_indices])
            points, labels = points[all_indices], labels[all_indices]
        elif num_points < self.target_size:
            # Increase number of points using nearest neighbor interpolation
            nbrs = NearestNeighbors(n_neighbors=2, algorithm='auto').fit(points)
            while points.shape[0] < self.target_size:
                idx = np.random.choice(num_points)
                point1 = points[idx]
                # Exclude the point itself using the second nearest neighbor
                point2 = points[nbrs.kneighbors([point1], 2)[1][0][1]]
                new_point = (point1 + point2) / 2
                new_label = labels[idx]
                points = np.vstack((points, new_point))
                labels = np.append(labels, new_label)
            points, labels = points[:self.target_size], labels[:self.target_size]
        return points, labels

    def normalize_points(self, points):
  
        mean = points.mean(axis=0)
        std = points.std(axis=0)
        return (points - mean) / (std + 1e-8)

    def __getitem__(self, idx):
        scan_id = self.files[idx]
        ply_file = os.path.join(self.root, scan_id, f"{scan_id}_vh_clean_2.labels.ply")
        plydata = PlyData.read(ply_file)
        vertex = plydata['vertex']
        
        # Get xyz coordinates and labels
        points = np.vstack([vertex['x'], vertex['y'], vertex['z']]).T  # (N, 3)
        labels = np.array(vertex['label'])  # (N,)
        
        # Apply transformation if available
        if self.transform:
            points, labels = self.resize_point_cloud(points, labels)
            # Construct data dictionary for transformation
            point_cloud_data = {'pos': points, 'labels': labels}
            point_cloud_data = self.transform(point_cloud_data)
            points = point_cloud_data['pos']
            labels = point_cloud_data['labels']
            # Rearrange tensor shape to (3, N)
            points = points.permute(1, 0)
            labels = torch.tensor(labels, dtype=torch.long)
        
        return points, labels

# Define training and testing transforms
train_transform = Compose([
    PointCloudToTensor(),
    PointCloudXYZAlign(gravity_dim=2),
    RandomHorizontalFlip(upright_axis='z', aug_prob=0.8),
    PointCloudScaling(scale=[0.8, 1.2]),
    PointCloudTranslation(shift=[0.2, 0.2, 0.2]),
    PointCloudJitter(jitter_sigma=0.01, jitter_clip=0.05),
    PointCloudCenterAndNormalize()
])

test_transform = Compose([
    PointCloudToTensor(),
    PointCloudCenterAndNormalize()
])

# AverageMeter class to track metrics over training/validation iterations
class AverageMeter:
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    
    def update(self, val, n=1):
        self.val = val  # Update current value
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# TNet: A transformation network to predict an affine transformation matrix
class TNet(nn.Module):
    def __init__(self, dim, num_points=10000):
        super().__init__()
        self.dim = dim
        self.layer1 = self._make_mlp_block(dim, 64)
        self.layer2 = self._make_mlp_block(64, 128)
        self.layer3 = self._make_mlp_block(128, 1024)
        self.max_pool = nn.MaxPool1d(num_points)
        self.layer4 = self._make_fc_block(1024, 512)
        self.layer5 = self._make_fc_block(512, 256)
        self.fc1 = nn.Linear(256, dim**2)
    
    def _make_mlp_block(self, in_dim, out_dim, kernel_size=1):

        return nn.Sequential(
            nn.Conv1d(in_dim, out_dim, kernel_size),
            nn.BatchNorm1d(out_dim),
            nn.ReLU()
        )
    
    def _make_fc_block(self, in_dim, out_dim):
  
        return nn.Sequential(
            nn.Linear(in_dim, out_dim),
            nn.BatchNorm1d(out_dim),
            nn.ReLU()
        )
    
    def forward(self, x):
        bs = x.shape[0]
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        # Global feature aggregation
        x = self.max_pool(x).view(bs, -1)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.fc1(x)
        # Add identity matrix for stability
        identity = torch.eye(self.dim, requires_grad=True).repeat(bs, 1, 1).to(x.device)
        x = x.view(-1, self.dim, self.dim) + identity
        return x

# PointNetBackbone extracts both local and global features
class PointNetBackbone(nn.Module):
    def __init__(self, num_points=10000, num_global_features=1024):
        super().__init__()
        self.num_points = num_points
        self.TNet_1 = TNet(dim=3, num_points=num_points)
        self.TNet_2 = TNet(dim=64, num_points=num_points)
        self.layer1 = self._make_conv_block(3, 64)
        self.layer2 = self._make_conv_block(64, 64)
        self.layer3 = self._make_conv_block(64, 64)
        self.layer4 = self._make_conv_block(64, 128)
        self.layer5 = self._make_conv_block(128, num_global_features)
        self.max_pool = nn.MaxPool1d(num_points, return_indices=True)
    
    def _make_conv_block(self, in_dim, out_dim, kernel_size=1):
 
        return nn.Sequential(
            nn.Conv1d(in_dim, out_dim, kernel_size),
            nn.BatchNorm1d(out_dim),
            nn.ReLU()
        )
    
    def forward(self, x):
        bs = x.shape[0]
        # Input transformation
        A_input = self.TNet_1(x)
        x = torch.bmm(x.transpose(2, 1), A_input).transpose(2, 1)
        x = self.layer1(x)
        x = self.layer2(x)
        # Feature transformation
        A_feat = self.TNet_2(x)
        x = torch.bmm(x.transpose(2, 1), A_feat).transpose(2, 1)
        local_features = x.clone()
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        # Global feature aggregation with indices
        global_features, critical_indexes = self.max_pool(x)
        global_features = global_features.view(bs, -1)
        critical_indexes = critical_indexes.view(bs, -1)
        # Concatenate local features with repeated global features for each point
        features = torch.cat((local_features, global_features.unsqueeze(-1).repeat(1, 1, self.num_points)), dim=1)
        return features, critical_indexes, A_feat

# PointNet model for segmentation
class PointNet(nn.Module):
    def __init__(self, num_points=10000, num_global_features=1024, num_classes=41):
        super().__init__()
        self.backbone = PointNetBackbone(num_points, num_global_features)
        # Combined features: global features + 64 (from early layers)
        num_features = num_global_features + 64
        self.layer1 = self._make_conv_block(num_features, 512)
        self.layer2 = self._make_conv_block(512, 256)
        self.layer3 = self._make_conv_block(256, 128)
        self.fc1 = nn.Conv1d(128, num_classes, 1)
    
    def _make_conv_block(self, in_dim, out_dim, kernel_size=1):
        """
        Create a convolutional block for segmentation head.
        """
        return nn.Sequential(
            nn.Conv1d(in_dim, out_dim, kernel_size),
            nn.BatchNorm1d(out_dim),
            nn.ReLU()
        )
    
    def forward(self, x):
        x, critical_indexes, A_feat = self.backbone(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.fc1(x)
        return x, critical_indexes, A_feat

# Function to plot training and validation metrics over epochs
def plot_segmentation_metrics(train_loss, val_loss, train_mean_iou=None, val_mean_iou=None, 
                              train_dice_score=None, val_dice_score=None):
    """
    Plot the training/validation loss, mean IoU and Dice score over epochs.
    """
    epochs = range(1, len(train_loss) + 1)
    plt.figure(figsize=(15, 10))
    
    # Plot Loss curves
    plt.subplot(2, 2, 1)
    plt.plot(epochs, train_loss, 'r', label='Training Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot Mean IoU curves if available
    if train_mean_iou is not None and val_mean_iou is not None:
        plt.subplot(2, 2, 2)
        plt.plot(epochs, train_mean_iou, 'r', label='Training Mean IoU')
        plt.plot(epochs, val_mean_iou, 'b', label='Validation Mean IoU')
        plt.title('Training and Validation Mean IoU')
        plt.xlabel('Epochs')
        plt.ylabel('Mean IoU')
        plt.legend()
    
    # Plot Dice score curves if available
    if train_dice_score is not None and val_dice_score is not None:
        plt.subplot(2, 2, 3)
        plt.plot(epochs, train_dice_score, 'r', label='Training Dice Score')
        plt.plot(epochs, val_dice_score, 'b', label='Validation Dice Score')
        plt.title('Training and Validation Dice Score')
        plt.xlabel('Epochs')
        plt.ylabel('Dice Score')
        plt.legend()
    
    plt.tight_layout()
    plt.show()

# Main training function
def train(num_class, n_workers, batch_size, lr, epochs, log_dir, saved_model):
    # Set device to GPU if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Prepare training and testing datasets
    train_dataset = ScannetDataset(transform=train_transform)
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=n_workers
    )

    test_dataset = ScannetDataset(is_train=False, transform=test_transform)
    test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=batch_size,
        num_workers=n_workers
    )
    
    checkpoint_path = "/kaggle/input/pointnet_best/pytorch/pointnet_best/1/best.pt"
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    model = PointNet(num_points=10000, num_global_features=1024, num_classes=num_class).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])

    # Define the loss function with label smoothing
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    # Initialize the Lion optimizer with a reduced learning rate
    optimizer = Lion(
        params=model.parameters(),
        lr=lr * 0.1,  # Lion requires a smaller learning rate
        weight_decay=1e-4
    )
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])  # Ensure compatibility with the checkpoint

    # Learning rate scheduler based on validation mean IoU
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.1, patience=1, verbose=True
    )
    
    cm = ConfusionMatrix(num_classes=num_class)
    
    # Create log and model save directories, removing them if they already exist
    if os.path.isdir(log_dir):
        shutil.rmtree(log_dir)
    os.makedirs(log_dir)
    
    if not os.path.isdir(saved_model):
        os.makedirs(saved_model)
    
    writer = SummaryWriter(log_dir)
    
    train_loss_history = []
    val_loss_history = []
    
    best_mIoU = -1
    no_improvement_counter = 0
    patience = 10  # Early stopping patience
    current_epoch = 0  # Start from epoch 0
    
    print(f"Starting training from epoch {current_epoch} to {epochs}")
    for epoch in range(current_epoch, epochs):
        model.train()
        train_loss_meter = AverageMeter()

        progress_bar = tqdm(train_loader, colour='cyan', desc=f"Training Epoch {epoch + 1}")
        for i, (features, masks) in enumerate(progress_bar):
            features, masks = features.to(device), masks.to(device)
            output, _, _ = model(features)
            loss = criterion(output, masks)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss_meter.update(loss.item())
            progress_bar.set_description(
                f"Train | Epoch {epoch + 1}/{epochs} | Loss: {train_loss_meter.avg:.4f} | LR: {optimizer.param_groups[0]['lr']:.6f}"
            )
            writer.add_scalar("Train/Loss", loss.item(), epoch * len(train_loader) + i)
        
        # Evaluate on the validation set
        model.eval()
        val_loss_meter = AverageMeter()
        cm.reset()
        
        with torch.no_grad():
            progress_bar_val = tqdm(test_loader, colour='yellow', desc="Validation")
            for features, masks in progress_bar_val:
                features, masks = features.to(device), masks.to(device)
                output, _, _ = model(features)
                loss = criterion(output, masks)
                predictions = torch.argmax(output, dim=1)
                cm.update(predictions, masks)
                val_loss_meter.update(loss.item())
                progress_bar_val.set_description(
                    f"Val | Loss: {loss.item():.4f} | Overall Acc: {cm.overall_accuray:.2f}"
                )
        
        miou, macc, overall_acc, _, _ = cm.all_metrics()
        print(f"Epoch {epoch + 1} | Val Loss: {val_loss_meter.avg:.4f} | mIoU: {miou:.4f} | OA: {overall_acc:.2f} | mACC: {macc:.2f}")
        writer.add_scalar("Val/Loss", val_loss_meter.avg, epoch)
        writer.add_scalar("Val/mIoU", miou, epoch)
        writer.add_scalar("Val/OA", overall_acc, epoch)
        
        # Update scheduler based on mean IoU
        scheduler.step(miou)
        
        # Save a temporary checkpoint after each epoch
        checkpoint_dict = {
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "epoch": epoch + 1,
            "mIoU": miou
        }
        torch.save(checkpoint_dict, os.path.join(saved_model, "new.pt"))
        
        train_loss_history.append(train_loss_meter.avg)
        val_loss_history.append(val_loss_meter.avg)
        
        # Save the best model based on mIoU
        if miou > best_mIoU:
            best_mIoU = miou
            torch.save(checkpoint_dict, os.path.join(saved_model, "best.pt"))
            no_improvement_counter = 0
        else:
            no_improvement_counter += 1
        
        if no_improvement_counter > patience:
            print("Early stopping triggered!")
            break

    print("Training Loss History:", train_loss_history)
    print("Validation Loss History:", val_loss_history)
    plot_segmentation_metrics(train_loss_history, val_loss_history)
    writer.close()

# Training hyperparameters and configuration
num_class = 41
n_workers = os.cpu_count()
batch_size = 32
lr = 1e-3
epochs = 50
log_dir = "/kaggle/working/tensorboard"
saved_model = "/kaggle/working/saved_model"

if __name__ == "__main__":
    train(num_class, n_workers, batch_size, lr, epochs, log_dir, saved_model)
